In [4]:
from ai_base import *

In [5]:
from collections import OrderedDict
from typing import List, Tuple

def _getOneHot(fdata: List[List], word_dict: OrderedDict) -> np.array:
    D = len(fdata)
    if len(word_dict) is 0:
        for row in fdata:
            for word in row:
                if not word in word_dict:
                    word_dict[word] = 1
        word_dict[None] = 0
    else:
        word_dict = dict(zip(word_dict.keys(), [0 for _ in word_dict.values()]))
        for row in fdata:
            for word in row:
                if word in word_dict:
                    word_dict[word] = 1
                else:
                    word_dict[None] = 1
    word_vec = word_dict.keys()
    word_order = dict(zip(word_vec,range(len(word_vec))))
    oneHot = np.zeros((D,len(word_dict)), dtype=np.int8)
    for i,row in enumerate(fdata):
        for word in row:
            if word in word_order:
                oneHot[i][word_order[word]] = 1
            else:
                oneHot[i][word_order[None]] = 1
    return oneHot
def OneHot(trainData: List[List], testData: List[List], savePath="")-> Tuple[np.array, np.array]:
    wdict = OrderedDict()
    trainX = _getOneHot(trainData, wdict)
    testX = _getOneHot(testData, wdict)
    if savePath != "-1":
        List2CSV(savePath + "onehot" + "TrainSet" + str(len(wdict)) + "D" + str(len(trainData)) + "L.csv", trainX)
        List2CSV(savePath + "onehot" + "TestSet" + str(len(wdict)) + "D" + str(len(testData)) + "L.csv", testData)
    return trainX, testX


In [3]:
trainData = CSV2List2('data/2/clean/trainDataclean.csv')
testData = CSV2List2('data/2/clean/testDataclean.csv')

In [4]:
X, _ = OneHot(trainData, testData, "-1")

In [6]:
X = np.float64(CSV2List2('doc2vecTrainSet50D24000L.csv'))
Y = np.uint8(CSV2List2('data/2/clean/trainLabel.txt'))
#数据集
D = np.c_[X, Y]


NameError: name 'X' is not defined

In [52]:
## D_parted[k] 对应分类为k的样本集
D_parted = list(range(K))
for k in range(2):
    ## 通过filter函数筛选
    D_parted[k] = np.array(list(filter(lambda v: v[-1] == k, D)))
    ## 计算先验概率
    P_Y[k] = len(D_parted[k]) / len(D)

In [53]:
P_Y

array([[0.65104167],
       [0.34895833]])

In [54]:
## 定义高斯分布类
class Guass:
    # 均值
    mean  = 0
    #标准差
    stdev = 0
    def __init__(self, stats):
        self.mean, self.stdev = stats
    # 计算x处的概率
    def P(self, x):
        t1 = np.exp(-(np.power(x-self.mean,2)/(2*np.power(self.stdev,2))))
        return (1 / (np.sqrt(2*np.pi) * self.stdev)) * t1

In [55]:
guass = list(range(K))
for k in range(K):
    # 统计每一维度特征在K种类别上的均值和方差，并依此构造高斯分布
    guass[k] = [Guass((np.mean(attr), np.std(attr))) for attr in zip(*D_parted[k])]
    del guass[k][-1]

In [18]:

onehotPXY = list(range(K))
for k in range(K):
    l = list(range(len(X[0]) + 1))
    z = zip(*D_parted[k])
    for i, attr in enumerate(tqdm_notebook(z)):
        l[i] = (1-(sum(attr)+1)/(len(D_parted[k])+2), (sum(attr)+1)/(len(D_parted[k])))
    del l[-1]
    onehotPXY[k] = l
        

In [30]:
List2CSV("NB_onehotPXY.csv", onehotPXY)

In [56]:
def split(s, t, X, Y):
    piece = int(N / s)
    vaildX = []
    vaildY = []
    if t == s - 1:
        vaildX = X[t * piece:]
        vaildY = Y[t * piece:]
    else:
        vaildX = X[t * piece: (t+1) * piece]
        vaildY = Y[t * piece: (t+1) * piece]
    return vaildX, vaildY

In [ ]:
import random
def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]
splitRatio = 0.67
train, test = splitDataset(dataset, splitRatio)

In [57]:
def predict(X):
    _PY = deepcopy(P_Y)
    for k in range(K):
        for i in range(M):
            ret = np.log(guass[k][i].P(X[i]))
            _PY[k] += ret
    return np.argmax(_PY)

In [20]:
def onehot_predict(X):
    _PY = deepcopy(P_Y)
    for k in range(K):
        for i in range(M):
            if X[i] == 1:
                ret = np.log(onehotPXY[k][i][X[i]])
                _PY[k] += ret
    return np.argmax(_PY)

In [28]:
import os
def onehot_predicts(Xs):
    print('Run task (%s)...' % (os.getpid()))
    return [onehot_predict(X) for X in Xs]

In [59]:
from tqdm import tnrange
from copy import deepcopy
for s in range(5, 6):
    acc = 0
    for t in range(1):
        vaildX, vaildY = split(s, t, X, Y)
        predictY = np.zeros_like(vaildY)
        list_multiprocess(vaildX, onehot_predicts, 12)
        correct = 0
        for i in range(len(predictY)):
            if(predictY[i] == vaildY[i]):
                correct += 1
        acc += correct / len(predictY)
    print(s, acc / s)

NameError: name 'onehot_predicts' is not defined

In [68]:
from tqdm import tnrange
from copy import deepcopy
for s in range(5, 6):
    acc = 0
    for t in range(s):
        vaildX, vaildY = split(s, t, X, Y)
        predictY = np.zeros_like(vaildY)
        for i in tnrange(len(predictY)):
            predictY[i] = predict(vaildX[i])
        correct = 0
        for i in range(len(predictY)):
            if(predictY[i] == vaildY[i]):
                correct += 1
        acc += correct / len(predictY)
    print(s, acc / s)


5 0.7512317747611865
